In [ ]:
!pip install openeo openeo-pg-parser-networkx openeo[localprocessing] openeo-processes-dask openeo-processes

In [1]:
from openeo.local import LocalConnection
from pathlib import Path
import pytest
import xarray as xr

from tensorlakehouse_openeo_driver.constants import DEFAULT_X_DIMENSION, DEFAULT_Y_DIMENSION


TypeError: unsupported operand type(s) for |: 'type' and 'types.GenericAlias'

In [ ]:
from openeo.local import LocalConnection
from pathlib import Path
import pytest
import xarray as xr

from tensorlakehouse_openeo_driver.constants import DEFAULT_X_DIMENSION, DEFAULT_Y_DIMENSION

# from tensorlakehouse_openeo_driver.local.connection import GeodnLocalConnection


@pytest.mark.parametrize(
    "path",
    [
        "/Users/ltizzei/Projects/Orgs/IBM/tensorlakehouse-openeo-driver/tensorlakehouse_openeo_driver/tests/test_data/test_openeo_Globalweather-ERA5-.nc",
        "/Users/ltizzei/Projects/Orgs/IBM/tensorlakehouse-openeo-driver/tensorlakehouse_openeo_driver/tests/test_data/tasmax_rcp85_land-cpm_uk_2.2km/tasmax_rcp85_land-cpm_uk_2.2km_01_day_20701201-20711130_reproj.nc",
    ],
)
def test_local_connection(path: str):
    p = Path(path)
    assert p.exists(), f"Error! File does not exist {p}"

    local_conn = LocalConnection(p.parent)

    # list_collections = local_conn.list_collections()
    coll = local_conn.describe_collection(path)
    cube_dimensions = coll["cube:dimensions"]
    temporal_dim_name = None
    for dim_name, dim_info in cube_dimensions.items():
        if dim_info.get("axis") is not None:
            if dim_info.get("axis") == DEFAULT_X_DIMENSION: 
                west = dim_info["extent"][0]
                east = dim_info["extent"][1]
            if dim_info.get("axis") == DEFAULT_Y_DIMENSION: 
                south = dim_info["extent"][0]
                north = dim_info["extent"][1]
        elif dim_info.get("type") == "temporal":
            temporal_dim_name = dim_name
    coords = list()
    for lat in [south, north]:
        for lon in [west, east]:
            coords.append([lon, lat])
    datacube = local_conn.load_collection(path)
    datacube = datacube.reduce_dimension(dimension=temporal_dim_name, reducer="mean")
    data: xr.DataArray = datacube.execute()
    assert isinstance(data, xr.DataArray)
    assert temporal_dim_name not in data.dims